### 라이브러리 불러오기

In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [3]:
#pandas와 numpy library import



### 데이터 불러오기

In [ ]:
titanic = 
titanic_test = 

In [ ]:
# titanic 자료 5개만 불러와보자


In [ ]:
# (rows, columns)


In [ ]:
# 수치자료에 대한 통계적 요약 정보 보기


In [ ]:
# 자료 전반에 대한 기본 정보 보기


In [ ]:
# null 값이 존재하는 컬럼 파악하기


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1)

### 데이터 시각화하기

In [7]:
#전체 변수에 대해 히스토그램 그려보기


In [ ]:
# 성별과 나이에 대해 히스토그램 그려보기


In [ ]:
# 생존여부와 Pclass, Age, Fare 간의 점도표 그려보기


In [ ]:
# 생존여부와 성별, Fare 간의 점도표 그려보기


In [ ]:
# 탑승지역(Embarked)별 승객수 분포 그려보기


In [ ]:
# 탑승지역(Embarked)별 생존여부 분포 그려보기


In [ ]:
# 성별, Pclass별 생존률 비교해보기


In [ ]:
# 나이에 따른 생존여부 분포 그려보기


In [ ]:
# 나이별 Pclass별 KDE(커널밀도추정)


In [ ]:
# 변수별 상관관계 파악해보기


In [ ]:
# 생존여부와 각 변수간 상관계수보기


### 결측값 보정하기

#### Embarked Column

In [ ]:
# null 값 보기


In [ ]:
# Fare 와 Pclass로 박스플랏 그려보기


In [ ]:
# 결측값 채워넣기


In [ ]:
# test 자료 결측값 있는지 살펴보기


#### Fare Column

In [ ]:
# null 값 보기


In [ ]:
# 중앙값으로 null 값 보정하기


##### Age Column

In [ ]:
with sns.plotting_context("notebook",font_scale=1.5):
    sns.set_style("whitegrid")
    sns.distplot(titanic["Age"].dropna(),
                 bins=80,
                 kde=False,
                 color="red")
    sns.plt.title("Age Distribution")
    plt.ylabel("Count")

In [ ]:
# Random Froest로 Missing Value 추정하여 채우기

from sklearn.ensemble import RandomForestRegressor

def fill_missing_age(df):
    
    #Feature set
    age_df = df[['Age','Embarked','Fare', 'Parch', 'SibSp',
                'Pclass','FamilySize',
                 'FsizeD','Deck']]
    # Split sets into train and test
    train  = age_df.loc[ (df.Age.notnull()) ]# known Age values
    test = age_df.loc[ (df.Age.isnull()) ]# null Ages
    
    # All age values are stored in a target array
    y = train.values[:, 0]
    
    # All the other values are stored in the feature array
    X = train.values[:, 1::]
    
    # Create and fit a model
    rtr = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
    rtr.fit(X, y)
    
    # Use the fitted model to predict the missing values
    predictedAges = rtr.predict(test.values[:, 1::])
    
    # Assign those predictions to the full data set
    df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 
    
    return df

In [ ]:
titanic=fill_missing_age(titanic)
titanic_test=fill_missing_age(titanic_test)

In [ ]:
with sns.plotting_context("notebook",font_scale=1.5):
    sns.set_style("whitegrid")
    sns.distplot(titanic["Age"].dropna(),
                 bins=80,
                 kde=False,
                 color="tomato")
    sns.plt.title("Age Distribution")
    plt.ylabel("Count")
    plt.xlim((0,80));

### Feature Engineering

In [ ]:
# Deck 변수를 바탕으로 실제 승객이 탄 장소를 분류하기


In [ ]:
# plot을 그려 유의미할 지 판단해보기


In [ ]:
# 결측값 보정하기


In [ ]:
# 가족수 추정하기


In [ ]:
# 추정 가족수를 기반으로 새로운 변수를 만들어보기


In [ ]:
# plot을 그려 유의미할 지 판단해보기


### Categorical 변수를 Numerical 변수로 변환하기
##### Embarked, Sex, FsizeD, Deck

### 보정된 자료를 바탕으로 생존여부와의 상관계수 보기

### 생존여부 예측하기
##### Linear Regression

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold

In [ ]:
# 사용할 변수와 타겟 변수 선정하기
predictors = [ ]

target =

# 알고리즘 선정하기
alg = LinearRegression()

# k겹 교차검증 (k-fold cross validation) 셋팅하기
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []

In [ ]:
# k-fold cross validation 기반 Linear Regression 실행하기
for train, test in kf:
    train_predictors = (titanic[predictors].iloc[train,:])
    train_target = titanic[target].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

In [ ]:
predictions = np.concatenate(predictions, axis=0)

# 예측률이 0.5 초과인 경우 생존으로, 0.5 이하인 경우 사망으로 분류
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0

# 테스트 데이터 내 전체 생존자수 중 모델로 맞춘 생존자 수 비율을 구해, 모델 정확도 측정하기
accuracy=sum(titanic["Survived"]==predictions)/len(titanic["Survived"])
accuracy